# 1

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!ls gdrive/MyDrive/datasets/ham10000

HAM10000_images_part_1	HAM10000_images_part_2	HAM10000_metadata.csv


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(0)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

In [4]:
import torch
import torchvision.datasets
from torchvision import transforms

In [5]:
X_train = torch.load('gdrive/MyDrive/datasets/ham_100_X_train.pt')
y_train = torch.load('gdrive/MyDrive/datasets/ham_100_y_train.pt')
X_test = torch.load('gdrive/MyDrive/datasets/ham_100_X_test.pt')
y_test = torch.load('gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [23]:
X_train /= 255.
X_test /= 255.

In [ ]:
X_train = torch.nn.functional.normalize(X_train)
X_test = torch.nn.functional.normalize(X_test)

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [15]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [6]:
import random

import torchvision.datasets
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [7]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

In [8]:
import pytorch_optimizer as optim

In [63]:
def train(net, X_train, y_train, X_test, y_test):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=3.0e-3)
    #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-4)
    optimizer = optim.PNMBelief(net.parameters(), lr=0.001)
    #optimizer = optim.YogiPNM(net.parameters())
    #optimizer = optim.AdaBelief(net.parameters())
    #optimizer = optim.DiffGrad(net.parameters(), lr=0.5e-3)
    #optimizer = optim.Yogi(net.parameters())
    
    batch_size = 10

    test_accuracy_history = []
    test_loss_history = []

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    for epoch in range(14):
        print("epoch ", epoch)
        order = np.random.permutation(len(X_train))
        
        if epoch == 9:
            #optimizer = torch.optim.SGD(net.parameters(), lr=1.0e-4)
            #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-5)
            optimizer = optim.PNMBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.YogiPNM(net.parameters(), lr=1.0e-4)
            #optimizer = optim.AdaBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.DiffGrad(net.parameters(), lr=1.0e-5)
            #optimizer = optim.Yogi(net.parameters(), lr=1.0e-4)
    

        for start_index in tqdm(range(0, len(X_train), batch_size)):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            X_batch

        net.eval()
        with torch.no_grad():
            test_preds = net.forward(X_test)
            loss_value = loss(test_preds, y_test).item()
            test_loss_history.append(loss_value)

            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().item()
            test_accuracy_history.append(accuracy)
            #print(test_preds.argmax(dim=1))
            #print(y_test)

        print("accuracy = ", accuracy)
        print("\n\n")

    return test_accuracy_history, test_loss_history

In [10]:
accuracies = {}
losses = {}

In [11]:
!pip install timm
import timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00


In [ ]:
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [12]:
from torchvision.models import alexnet, AlexNet_Weights



In [26]:
model = torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)


accuracies['AlexNet+SGD'], losses['AlexNet+SGD'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:09<00:00, 88.95it/s]


accuracy =  0.7129306197166443



epoch  1


100%|██████████| 802/802 [00:08<00:00, 89.54it/s]


accuracy =  0.7304043769836426



epoch  2


...

accuracy =  0.7878182530403137



epoch  12


100%|██████████| 802/802 [00:09<00:00, 87.31it/s]


accuracy =  0.7923115491867065



epoch  13


100%|██████████| 802/802 [00:09<00:00, 87.11it/s]


accuracy =  0.7918123006820679





In [40]:
model = torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)



accuracies['AlexNet+Adam'], losses['AlexNet+Adam'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:22<00:00, 34.91it/s]


accuracy =  0.7348976731300354



epoch  1


100%|██████████| 802/802 [00:22<00:00, 34.88it/s]


accuracy =  0.7294058799743652



epoch  2


...

accuracy =  0.8117823004722595



epoch  12


100%|██████████| 802/802 [00:23<00:00, 34.57it/s]


accuracy =  0.8087868094444275



epoch  13


100%|██████████| 802/802 [00:23<00:00, 34.60it/s]


accuracy =  0.8107838034629822





In [48]:
model = torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)


accuracies['AlexNet+Yogi'], losses['AlexNet+Yogi'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:25<00:00, 31.34it/s]


accuracy =  0.7174238562583923



epoch  1


100%|██████████| 802/802 [00:25<00:00, 31.32it/s]


accuracy =  0.6789815425872803



epoch  2


...

accuracy =  0.7703444957733154



epoch  12


100%|██████████| 802/802 [00:25<00:00, 31.24it/s]


accuracy =  0.7688467502593994



epoch  13


100%|██████████| 802/802 [00:25<00:00, 31.24it/s]


accuracy =  0.7748377323150635





In [56]:
model = torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)


accuracies['AlexNet+DiffGrad'], losses['AlexNet+DiffGrad'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:37<00:00, 21.44it/s]


accuracy =  0.7353969216346741



epoch  1


100%|██████████| 802/802 [00:37<00:00, 21.56it/s]


accuracy =  0.7009485960006714



epoch  2


...

accuracy =  0.8002995252609253



epoch  12


100%|██████████| 802/802 [00:37<00:00, 21.56it/s]


accuracy =  0.800798773765564



epoch  13


100%|██████████| 802/802 [00:37<00:00, 21.56it/s]


accuracy =  0.8017973303794861





In [64]:
model = torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)


accuracies['AlexNet+PNMBelief'], losses['AlexNet+PNMBelief'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:36<00:00, 22.13it/s]


accuracy =  0.5167248845100403



epoch  1


100%|██████████| 802/802 [00:36<00:00, 22.13it/s]


accuracy =  0.4717923104763031



epoch  2


...

accuracy =  0.7963055372238159



epoch  12


100%|██████████| 802/802 [00:36<00:00, 22.10it/s]


accuracy =  0.7913130521774292



epoch  13


100%|██████████| 802/802 [00:36<00:00, 22.08it/s]


accuracy =  0.7898153066635132





# ansamble

In [27]:
APreds = pd.DataFrame()

In [65]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [29]:
APreds['SGD'] = test_preds.argmax(dim=1).tolist()

In [42]:
APreds['Adam'] = test_preds.argmax(dim=1).tolist()

In [50]:
APreds['Yogi'] = test_preds.argmax(dim=1).tolist()

In [58]:
APreds['DiffGrad'] = test_preds.argmax(dim=1).tolist()

In [66]:
APreds['PNMBelief'] = test_preds.argmax(dim=1).tolist()

In [67]:
APreds

,SGD,Adam,Yogi,DiffGrad,PNMBelief
0,4,4,4,4,4
1,4,4,4,4,4
2,4,4,4,4,4
3,2,2,2,2,2
4,4,4,1,2,1
...,...,...,...,...,...
1998,4,4,4,4,4
1999,4,5,5,5,4
2000,4,4,4,4,4
2001,4,4,4,4,4


In [ ]:
APreds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_resnext.csv')
APreds = APreds.drop(['Unnamed: 0'], axis=1)

In [ ]:
APreds.to_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_resnext.csv')

In [73]:
ARes = pd.DataFrame(np.zeros((2003,7), dtype=int))

for i in range(2003):
    for j in range(5):
        if j == 0:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.7918
        if j == 1:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.81078
        if j == 2:
            ARes.iloc[i, APreds.iloc[i, j]] += 0 #0.7748
        if j == 3:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.80179
        if j == 4:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.7898

In [74]:
(torch.tensor(ARes.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8142786026000977

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(test_preds.argmax(dim=1).tolist(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #Adam

In [ ]:
D #PNMBelief

In [ ]:
D #Yogi

In [ ]:
D #DiffGrad

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(ARes.idxmax(axis=1).to_list(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #ansamble

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
precision_score(y_test, test_preds.argmax(dim=1), average='micro')


In [ ]:
print('precision: ', precision_score(y_test, test_preds.argmax(dim=1), average=None))
print('recall: ', recall_score(y_test, test_preds.argmax(dim=1), average=None))
print('f1_score: ', f1_score(y_test, test_preds.argmax(dim=1), average=None))

# SoftMax

In [31]:
ARes2 = pd.DataFrame(np.zeros((2003,7), dtype=int))

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [33]:
m = torch.nn.Softmax(dim=1)

In [34]:
test_preds = m(test_preds) * 0.7918 #SGD

In [44]:
test_preds = m(test_preds) * 0.81078 #Adam

In [52]:
test_preds = m(test_preds) * 0.7748 #Yogi

In [60]:
test_preds = m(test_preds) * 0.80179 #DiffGrad

In [70]:
test_preds = m(test_preds) * 0.7898 #PNMBelief

In [71]:
ARes2 += pd.DataFrame(test_preds.to('cpu'))

In [ ]:
test_preds

In [37]:
ARes2

,0,1,2,3,4,5,6
0,1.628111e-07,4.113596e-07,0.000155,0.000019,0.791595,0.000030,1.042898e-06
1,1.248930e-05,3.966931e-06,0.011415,0.000093,0.775952,0.004301,9.526380e-06
2,3.750236e-05,1.109214e-04,0.001960,0.000036,0.661662,0.127983,1.779633e-06
3,5.756337e-02,9.988315e-03,0.633987,0.006660,0.040503,0.042949,4.397712e-05
4,4.207008e-02,7.526793e-02,0.181641,0.011315,0.457913,0.016917,1.641852e-04
...,...,...,...,...,...,...,...
1998,1.075976e-07,2.672156e-07,0.000386,0.000001,0.783783,0.007628,6.416328e-07
1999,1.940103e-04,8.764557e-05,0.031598,0.000032,0.532734,0.226980,1.672739e-04
2000,1.123166e-03,8.250599e-04,0.031384,0.006767,0.751218,0.000464,9.844050e-06
2001,3.145197e-05,4.052594e-04,0.026648,0.000075,0.747248,0.017349,2.226004e-05


In [72]:
(torch.tensor(ARes2.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.813779354095459